## 1. Importación de Librerías

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import cv2
from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 2. Configuración de Rutas

In [ ]:
ruta_base = Path(r'e:\06. Sexto Ciclo\01. Machine Learning\07. Workspace\16S03. Proyecto 03\P3-EcoSort')
ruta_train = ruta_base / 'data' / 'preprocessed' / 'train'
ruta_val = ruta_base / 'data' / 'preprocessed' / 'val'
ruta_features = ruta_base / 'result' / 'features'
ruta_figuras = ruta_base / 'result' / 'figures'

ruta_features.mkdir(parents=True, exist_ok=True)
ruta_figuras.mkdir(parents=True, exist_ok=True)

clases = ['general', 'paper', 'plastic']
mapeo_clases = {'general': 0, 'paper': 1, 'plastic': 2}
tamanio_imagen = (128, 128)

## 3. Carga y Normalización de Imágenes

In [ ]:
def cargar_imagenes(ruta, tamanio):
    imagenes = []
    etiquetas = []
    rutas_archivos = []
    
    for clase in clases:
        ruta_clase = ruta / clase
        archivos = list(ruta_clase.glob('*'))
        
        for archivo in tqdm(archivos, desc=f'Cargando {clase}'):
            if archivo.is_file():
                try:
                    img = Image.open(archivo)
                    img = img.convert('RGB')
                    img = img.resize(tamanio)
                    img_array = np.array(img) / 255.0
                    
                    imagenes.append(img_array)
                    etiquetas.append(mapeo_clases[clase])
                    rutas_archivos.append(str(archivo))
                except Exception as e:
                    pass
    
    return np.array(imagenes), np.array(etiquetas), rutas_archivos

X_train, y_train, rutas_train = cargar_imagenes(ruta_train, tamanio_imagen)
X_val, y_val, rutas_val = cargar_imagenes(ruta_val, tamanio_imagen)

In [ ]:
df_carga = pd.DataFrame({
    'Conjunto': ['Train', 'Validación'],
    'Cantidad Imágenes': [len(X_train), len(X_val)],
    'Dimensiones': [X_train.shape, X_val.shape],
    'Rango Valores': [f'[{X_train.min():.2f}, {X_train.max():.2f}]', 
                      f'[{X_val.min():.2f}, {X_val.max():.2f}]']
})

df_carga

## 4. Data Augmentation

### 4.1 Funciones de Augmentation

In [ ]:
def rotar_imagen(img, angulo):
    h, w = img.shape[:2]
    centro = (w // 2, h // 2)
    matriz = cv2.getRotationMatrix2D(centro, angulo, 1.0)
    return cv2.warpAffine(img, matriz, (w, h))

def voltear_horizontal(img):
    return cv2.flip(img, 1)

def voltear_vertical(img):
    return cv2.flip(img, 0)

def ajustar_brillo(img, factor):
    img_ajustada = img * factor
    return np.clip(img_ajustada, 0, 1)

def ajustar_contraste(img, factor):
    media = np.mean(img)
    img_ajustada = (img - media) * factor + media
    return np.clip(img_ajustada, 0, 1)

def aplicar_zoom(img, factor):
    h, w = img.shape[:2]
    nuevo_h, nuevo_w = int(h * factor), int(w * factor)
    img_zoom = cv2.resize(img, (nuevo_w, nuevo_h))
    
    if factor > 1:
        inicio_h = (nuevo_h - h) // 2
        inicio_w = (nuevo_w - w) // 2
        return img_zoom[inicio_h:inicio_h+h, inicio_w:inicio_w+w]
    else:
        resultado = np.zeros_like(img)
        inicio_h = (h - nuevo_h) // 2
        inicio_w = (w - nuevo_w) // 2
        resultado[inicio_h:inicio_h+nuevo_h, inicio_w:inicio_w+nuevo_w] = img_zoom
        return resultado

def trasladar_imagen(img, dx, dy):
    h, w = img.shape[:2]
    matriz = np.float32([[1, 0, dx], [0, 1, dy]])
    return cv2.warpAffine(img, matriz, (w, h))

### 4.2 Generación de Dataset Aumentado

In [ ]:
def augmentar_dataset(X, y, objetivo=10000):
    X_aug = list(X)
    y_aug = list(y)
    
    transformaciones = [
        lambda img: rotar_imagen(img, 15),
        lambda img: rotar_imagen(img, -15),
        lambda img: rotar_imagen(img, 30),
        lambda img: voltear_horizontal(img),
        lambda img: voltear_vertical(img),
        lambda img: ajustar_brillo(img, 1.2),
        lambda img: ajustar_brillo(img, 0.8),
        lambda img: ajustar_contraste(img, 1.3),
        lambda img: ajustar_contraste(img, 0.7),
        lambda img: aplicar_zoom(img, 1.2),
        lambda img: aplicar_zoom(img, 0.8),
        lambda img: trasladar_imagen(img, 10, 10),
        lambda img: trasladar_imagen(img, -10, -10)
    ]
    
    while len(X_aug) < objetivo:
        idx = np.random.randint(0, len(X))
        img = X[idx]
        etiqueta = y[idx]
        
        num_trans = np.random.randint(1, 4)
        trans_seleccionadas = np.random.choice(transformaciones, num_trans, replace=False)
        
        img_aug = img.copy()
        for trans in trans_seleccionadas:
            img_aug = trans(img_aug)
        
        X_aug.append(img_aug)
        y_aug.append(etiqueta)
        
        if len(X_aug) % 1000 == 0:
            tqdm.write(f'Imágenes aumentadas: {len(X_aug)}/{objetivo}')
    
    return np.array(X_aug), np.array(y_aug)

X_train_aug, y_train_aug = augmentar_dataset(X_train, y_train, objetivo=10000)

In [ ]:
df_augmentation = pd.DataFrame({
    'Conjunto': ['Original Train', 'Aumentado Train', 'Validación'],
    'Cantidad': [len(X_train), len(X_train_aug), len(X_val)],
    'Incremento': ['Base', f'+{len(X_train_aug) - len(X_train)}', 'Sin cambios']
})

df_augmentation

### 4.3 Visualización de Augmentation

In [ ]:
idx_ejemplo = 0
img_original = X_train[idx_ejemplo]

ejemplos = [
    ('Original', img_original),
    ('Rotación 15°', rotar_imagen(img_original, 15)),
    ('Volteo Horizontal', voltear_horizontal(img_original)),
    ('Brillo +20%', ajustar_brillo(img_original, 1.2)),
    ('Contraste +30%', ajustar_contraste(img_original, 1.3)),
    ('Zoom 1.2x', aplicar_zoom(img_original, 1.2))
]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Ejemplos de Data Augmentation', fontsize=16, fontweight='bold')

for idx, (titulo, img) in enumerate(ejemplos):
    ax = axes[idx // 3, idx % 3]
    ax.imshow(img)
    ax.set_title(titulo, fontsize=12, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.savefig(ruta_figuras / '02_fe_01_augmentation_ejemplos.svg', format='svg', bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

conteo_original = np.bincount(y_train)
conteo_aumentado = np.bincount(y_train_aug)

x = np.arange(len(clases))
ancho_barra = 0.35

axes[0].bar(x - ancho_barra/2, conteo_original, ancho_barra, label='Original', alpha=0.8, color='#3498db', edgecolor='black')
axes[0].bar(x + ancho_barra/2, conteo_aumentado, ancho_barra, label='Aumentado', alpha=0.8, color='#2ecc71', edgecolor='black')
axes[0].set_xlabel('Clase', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Cantidad', fontsize=12, fontweight='bold')
axes[0].set_title('Comparación: Original vs Aumentado', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(clases)
axes[0].legend(fontsize=10)
axes[0].grid(axis='y', alpha=0.3)

porcentajes_aumentado = conteo_aumentado / conteo_aumentado.sum() * 100
colores = ['#2ecc71', '#e74c3c', '#3498db']
axes[1].pie(porcentajes_aumentado, labels=clases, autopct='%1.1f%%', startangle=90, 
           colors=colores, textprops={'fontsize': 12, 'fontweight': 'bold'})
axes[1].set_title('Distribución Dataset Aumentado', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '02_fe_02_distribucion_aumentada.svg', format='svg', bbox_inches='tight')
plt.show()

## 5. Extracción de Características

### 5.1 Extracción HOG

In [ ]:
def extraer_hog(imagenes):
    caracteristicas_hog = []
    
    for img in tqdm(imagenes, desc='Extrayendo HOG'):
        img_gris = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
        
        features = hog(img_gris, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), visualize=False, feature_vector=True)
        
        caracteristicas_hog.append(features)
    
    return np.array(caracteristicas_hog)

hog_train = extraer_hog(X_train_aug)
hog_val = extraer_hog(X_val)

In [ ]:
df_hog = pd.DataFrame({
    'Conjunto': ['Train Aumentado', 'Validación'],
    'Dimensión HOG': [hog_train.shape, hog_val.shape],
    'Características por Imagen': [hog_train.shape[1], hog_val.shape[1]]
})

df_hog

### 5.2 Visualización de HOG

In [ ]:
idx_vis = 0
img_ejemplo = X_train_aug[idx_vis]
img_gris = cv2.cvtColor((img_ejemplo * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)

features, hog_image = hog(img_gris, orientations=9, pixels_per_cell=(8, 8),
                         cells_per_block=(2, 2), visualize=True, feature_vector=True)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img_ejemplo)
axes[0].set_title('Imagen Original (RGB)', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(img_gris, cmap='gray')
axes[1].set_title('Imagen en Escala de Grises', fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(hog_image, cmap='gray')
axes[2].set_title('Visualización HOG', fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.savefig(ruta_figuras / '02_fe_03_hog_visualizacion.svg', format='svg', bbox_inches='tight')
plt.show()

### 5.3 Extracción de Características de Color

In [ ]:
def extraer_caracteristicas_color(imagenes):
    caracteristicas_color = []
    
    for img in tqdm(imagenes, desc='Extrayendo Color'):
        img_uint8 = (img * 255).astype(np.uint8)
        
        hist_r = cv2.calcHist([img_uint8], [0], None, [32], [0, 256]).flatten()
        hist_g = cv2.calcHist([img_uint8], [1], None, [32], [0, 256]).flatten()
        hist_b = cv2.calcHist([img_uint8], [2], None, [32], [0, 256]).flatten()
        
        img_hsv = cv2.cvtColor(img_uint8, cv2.COLOR_RGB2HSV)
        hist_h = cv2.calcHist([img_hsv], [0], None, [32], [0, 180]).flatten()
        hist_s = cv2.calcHist([img_hsv], [1], None, [32], [0, 256]).flatten()
        hist_v = cv2.calcHist([img_hsv], [2], None, [32], [0, 256]).flatten()
        
        media_r, media_g, media_b = np.mean(img, axis=(0, 1))
        std_r, std_g, std_b = np.std(img, axis=(0, 1))
        
        momentos_color = [media_r, media_g, media_b, std_r, std_g, std_b]
        
        features = np.concatenate([hist_r, hist_g, hist_b, hist_h, hist_s, hist_v, momentos_color])
        caracteristicas_color.append(features)
    
    return np.array(caracteristicas_color)

color_train = extraer_caracteristicas_color(X_train_aug)
color_val = extraer_caracteristicas_color(X_val)

In [ ]:
df_color = pd.DataFrame({
    'Conjunto': ['Train Aumentado', 'Validación'],
    'Dimensión Color': [color_train.shape, color_val.shape],
    'Características por Imagen': [color_train.shape[1], color_val.shape[1]]
})

df_color

### 5.4 Fusión de Características

In [ ]:
features_train = np.concatenate([hog_train, color_train], axis=1)
features_val = np.concatenate([hog_val, color_val], axis=1)

df_fusion = pd.DataFrame({
    'Conjunto': ['Train Aumentado', 'Validación'],
    'Dimensión HOG': [hog_train.shape[1], hog_val.shape[1]],
    'Dimensión Color': [color_train.shape[1], color_val.shape[1]],
    'Dimensión Total': [features_train.shape[1], features_val.shape[1]],
    'Shape Final': [features_train.shape, features_val.shape]
})

df_fusion

## 6. Reducción Dimensional con PCA

In [ ]:
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_val_scaled = scaler.transform(features_val)

pca = PCA(n_components=0.95)
features_train_pca = pca.fit_transform(features_train_scaled)
features_val_pca = pca.transform(features_val_scaled)

varianza_explicada = pca.explained_variance_ratio_
varianza_acumulada = np.cumsum(varianza_explicada)

In [ ]:
df_pca = pd.DataFrame({
    'Métrica': ['Componentes Originales', 'Componentes PCA', 'Reducción (%)', 
                'Varianza Explicada (%)', 'Primera Componente (%)'],
    'Valor': [
        features_train.shape[1],
        features_train_pca.shape[1],
        ((features_train.shape[1] - features_train_pca.shape[1]) / features_train.shape[1] * 100),
        (varianza_acumulada[-1] * 100),
        (varianza_explicada[0] * 100)
    ]
})

df_pca['Valor'] = df_pca['Valor'].round(2)
df_pca

### 6.1 Visualización de PCA

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].plot(range(1, len(varianza_explicada) + 1), varianza_acumulada * 100, 
            marker='o', linestyle='-', linewidth=2, markersize=4, color='#3498db')
axes[0].axhline(y=95, color='#e74c3c', linestyle='--', linewidth=2, label='95% Varianza')
axes[0].axvline(x=features_train_pca.shape[1], color='#2ecc71', linestyle='--', 
               linewidth=2, label=f'Componentes: {features_train_pca.shape[1]}')
axes[0].set_xlabel('Número de Componentes', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Varianza Explicada Acumulada (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Varianza Explicada por PCA', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

top_n = 20
axes[1].bar(range(1, top_n + 1), varianza_explicada[:top_n] * 100, 
           alpha=0.8, color='#9b59b6', edgecolor='black')
axes[1].set_xlabel('Componente Principal', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Varianza Explicada (%)', fontsize=12, fontweight='bold')
axes[1].set_title(f'Top {top_n} Componentes Principales', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(ruta_figuras / '02_fe_04_pca_varianza.svg', format='svg', bbox_inches='tight')
plt.show()

### 6.2 Visualización 2D de PCA

In [ ]:
pca_2d = PCA(n_components=2)
features_train_2d = pca_2d.fit_transform(features_train_scaled)

fig, ax = plt.subplots(figsize=(12, 8))

colores_scatter = ['#2ecc71', '#e74c3c', '#3498db']
for idx, clase in enumerate(clases):
    mascara = y_train_aug == idx
    ax.scatter(features_train_2d[mascara, 0], features_train_2d[mascara, 1], 
              c=colores_scatter[idx], label=clase, alpha=0.6, s=30, edgecolors='black', linewidth=0.5)

ax.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}% varianza)', 
             fontsize=12, fontweight='bold')
ax.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}% varianza)', 
             fontsize=12, fontweight='bold')
ax.set_title('Visualización 2D de Características con PCA', fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(ruta_figuras / '02_fe_05_pca_2d.svg', format='svg', bbox_inches='tight')
plt.show()

## 7. Validación con Logistic Regression

In [ ]:
modelo_logistic = LogisticRegression(max_iter=1000, random_state=42)
modelo_logistic.fit(features_train_pca, y_train_aug)

y_pred_train = modelo_logistic.predict(features_train_pca)
y_pred_val = modelo_logistic.predict(features_val_pca)

accuracy_train = accuracy_score(y_train_aug, y_pred_train)
accuracy_val = accuracy_score(y_val, y_pred_val)

In [ ]:
df_validacion = pd.DataFrame({
    'Conjunto': ['Train', 'Validación'],
    'Accuracy': [accuracy_train, accuracy_val]
})

df_validacion['Accuracy'] = df_validacion['Accuracy'].round(4)
df_validacion

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val, y_pred_val)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=clases, yticklabels=clases, 
           cbar_kws={'label': 'Cantidad'}, linewidths=2, linecolor='black', ax=ax)
ax.set_xlabel('Predicción', fontsize=12, fontweight='bold')
ax.set_ylabel('Valor Real', fontsize=12, fontweight='bold')
ax.set_title(f'Matriz de Confusión - Validación Logistic Regression\nAccuracy: {accuracy_val:.4f}', 
            fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(ruta_figuras / '02_fe_06_confusion_matrix.svg', format='svg', bbox_inches='tight')
plt.show()

## 8. Almacenamiento de Características

In [ ]:
np.save(ruta_features / 'X_train_imagenes.npy', X_train_aug)
np.save(ruta_features / 'y_train.npy', y_train_aug)
np.save(ruta_features / 'X_val_imagenes.npy', X_val)
np.save(ruta_features / 'y_val.npy', y_val)

np.save(ruta_features / 'features_train_hog.npy', hog_train)
np.save(ruta_features / 'features_val_hog.npy', hog_val)

np.save(ruta_features / 'features_train_color.npy', color_train)
np.save(ruta_features / 'features_val_color.npy', color_val)

np.save(ruta_features / 'features_train_combinadas.npy', features_train)
np.save(ruta_features / 'features_val_combinadas.npy', features_val)

np.save(ruta_features / 'features_train_pca.npy', features_train_pca)
np.save(ruta_features / 'features_val_pca.npy', features_val_pca)

import joblib
joblib.dump(scaler, ruta_features / 'scaler.pkl')
joblib.dump(pca, ruta_features / 'pca.pkl')

In [ ]:
archivos_guardados = list(ruta_features.glob('*'))

df_archivos = pd.DataFrame({
    'Archivo': [f.name for f in archivos_guardados],
    'Tamaño (MB)': [(f.stat().st_size / (1024 * 1024)) for f in archivos_guardados]
})

df_archivos['Tamaño (MB)'] = df_archivos['Tamaño (MB)'].round(2)
df_archivos = df_archivos.sort_values('Tamaño (MB)', ascending=False)
df_archivos

## 9. Resumen de Feature Engineering

### Proceso Completado:

**1. Normalización:**
- Todas las imágenes redimensionadas a 128×128 píxeles
- Valores normalizados al rango [0, 1]
- Conversión uniforme a RGB

**2. Data Augmentation:**
- Dataset aumentado a más de 10,000 imágenes de entrenamiento
- Transformaciones aplicadas: rotación, volteo, brillo, contraste, zoom, traslación
- Distribución balanceada entre clases

**3. Extracción de Características:**
- **HOG**: Captura gradientes y formas en escala de grises
- **Color**: Histogramas RGB y HSV, más momentos estadísticos
- **Fusión**: Combinación completa de HOG + Color

**4. Reducción Dimensional:**
- PCA aplicado preservando 95% de varianza
- Reducción significativa de dimensiones
- Características listas para SVM y Logistic Regression

**5. Validación:**
- Logistic Regression simple confirma que las características son discriminativas
- Accuracy de validación indica separabilidad entre clases

**6. Almacenamiento:**
- Todas las matrices de características guardadas en result/features/
- Listas para uso directo en entrenamiento de modelos
- Scaler y PCA guardados para reproducibilidad